# Homework 3 – Deployment

## Tasks

![Iris flower](./data/iris.png)

* Import Iris dataset from `./data/iris.csv`.
* Create an endpoint `/train` which expects `n_estimators` and `max_depth` attributes of a Random Forest Classifier, trains a model using a Cross Validation, and returns the F1-Score, Precision, Recall and Accuracy.
* Create an endpoint `/predict` which expects the four input features (sepal length, sepal width, petal length, petal width) and predicts as well as retuns the iris class using the model from above.

## Please make sure that the *entire* notebook runs without any errors!
## Also, please use the predefined routes with their parameters. 

In [38]:
from flask import Flask, jsonify, request
app = Flask(__name__)

import sys

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score
from sklearn.metrics import make_scorer, SCORERS
from sklearn.preprocessing import LabelEncoder

In [39]:
iris = pd.read_csv("./data/iris.csv",index_col=0)
X = iris.iloc[:,:-1]
y = iris['target']

iris.head()

y.value_counts()

Iris-virginica     41
Iris-versicolor    36
Iris-setosa        34
Name: target, dtype: int64

In [40]:
rf_clf = RandomForestClassifier()
le = LabelEncoder()

In [41]:
"""
Return the scores with the provided naming convention.
Placehorder for your comments:

"""
@app.route('/train')
def train():
    p_n_estimators = int(request.args.get('n_estimators'))
    p_max_depth = int(request.args.get('max_depth'))
    #Code your solution here
    
        
    iris = pd.read_csv("./data/iris.csv",index_col=0)
    X = iris.iloc[:,:-1]
    y = iris['target']
    y_encoded = le.fit_transform(y)

    # set params from bayesian search
    params = {'n_estimators': p_n_estimators, 'max_depth': p_max_depth}
    rf_clf.set_params(**params)
    rf_clf.fit(X,y_encoded)

    scoring=('f1_weighted', 'precision_weighted','recall_weighted', 'accuracy')
    scores = cross_validate(rf_clf, X, y_encoded, n_jobs=-1, cv=3, scoring=scoring)
    
    #Paste the matching scores into the fields by replacing the respective ""
    return jsonify({
        'f1_score': f"{scores['test_f1_weighted'].mean()}",
        'precision': f"{scores['test_precision_weighted'].mean()}",
        'recall': f"{scores['test_recall_weighted'].mean()}",
        'accuracy': f"{scores['test_accuracy'].mean()}",
    })


In [42]:
"""
Keep the url argument names.
sl = sepal length
sw = sepal width
pl = petal length
pw = petal width

Also make sure that the returned flowername is matching the naming convention from the dataset. 
Placehorder for your comments:

"""
@app.route('/predict')
def predict():
    sl = float(request.args.get('sl'))
    sw = float(request.args.get('sw'))
    pl = float(request.args.get('pl'))
    pw = float(request.args.get('pw'))
    #Code your solution here
    
    query = np.array([sl,sw,pl,pw])
    query = query.reshape((1,-1))
    
    prediction = rf_clf.predict(query)

    #Return the predicted flower name as string
    return le.inverse_transform(prediction)[0]

**Train**

`get http://localhost:5000/train?n_estimators=100&max_depth=10`

**Predict**

`get http://localhost:5000/predict?sl=4.9&sw=3.0&pl=1.4&pw=0.2`

In [44]:
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
{'fit_time': array([0.16702533, 0.14411831, 0.14806175]), 'score_time': array([0.01603985, 0.01994205, 0.01400161]), 'test_f1_weighted': array([0.97297297, 0.89189189, 0.91903904]), 'test_precision_weighted': array([0.97505198, 0.89189189, 0.92099792]), 'test_recall_weighted': array([0.97297297, 0.89189189, 0.91891892]), 'test_accuracy': array([0.97297297, 0.89189189, 0.91891892])}
127.0.0.1 - - [25/Nov/2021 15:02:04] "GET /train?n_estimators=100&max_depth=10 HTTP/1.1" 200 -
127.0.0.1 - - [25/Nov/2021 15:02:04] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\Markus\anaconda3\envs\aiss\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [25/Nov/2021 15:02:32] "GET /predict?sl=4.9&sw=3.0&pl=1.4&pw=0.2 HTTP/1.1" 200 -
